In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/kaggle/input/poetry-data-set/Roman-Urdu-Poetry.csv',usecols=[2],names=["text_cleaned"])
data

,text_cleaned
0,Poetry
1,aañkh se duur na ho dil se utar jā.egā \nvaqt ...
2,āshiqī meñ 'mīr' jaise ḳhvāb mat dekhā karo \n...
3,ab aur kyā kisī se marāsim baḌhā.eñ ham \nye b...
4,ab ke ham bichhḌe to shāyad kabhī ḳhvāboñ meñ ...
...,...
1310,vo mere ghar nahīñ aatā maiñ us ke ghar nahīñ ...
1311,vo mujh ko kyā batānā chāhtā hai \njo duniyā s...
1312,ye hai to sab ke liye ho ye zid hamārī hai \ni...
1313,zarā sā qatra kahīñ aaj agar ubhartā hai \nsam...


# Tokenisation

In [3]:
from collections import Counter



# Tokenising into Characters
characters = list("".join(data['text_cleaned']))

# Create Vocab
vocab = Counter(characters)
vocab=sorted(vocab, key=vocab.get, reverse=True)
vocab_to_int={characters:i for i,characters in enumerate(vocab)}
int_to_vocab={i:characters for characters,i in vocab_to_int.items()}



# converting characters to integers
encoded_char=[vocab_to_int[char] for char in characters]


# Converting data into Input and output sequences
seq_length=102
input_sequences=[]
output_sequences=[]

for i in range(len(encoded_char)-seq_length):
  input_seq=encoded_char[i:i+seq_length]
  output_seq=encoded_char[i+seq_length]
  input_sequences.append(input_seq)
  output_sequences.append(output_seq)




# Converting into Tensors

In [4]:
import torch
input_sequences=torch.tensor(input_sequences,dtype=torch.long)
output_sequences=torch.tensor(output_sequences,dtype=torch.long)

# Making DataLoader

In [5]:
batch_size=256
from torch.utils.data import Dataset,DataLoader
class PoemDataset(Dataset):
  def __init__(self,input_sequences,output_sequences):
    self.input_sequences=input_sequences
    self.output_sequences=output_sequences

  def __len__(self):
    return len(self.input_sequences)

  def __getitem__(self,idx):
    return self.input_sequences[idx],self.output_sequences[idx]

dataset=PoemDataset(input_sequences,output_sequences)

train_loader=DataLoader(dataset,batch_size=256,shuffle=True)


# Build GRU Model

In [6]:
import torch.nn as nn


class GRUPoemGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super(GRUPoemGenerator, self).__init__()
        self.hidden_size = hidden_size  # <-- Add this line
        self.num_layers = num_layers  # If using multiple layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        print(self.gru)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.gru(x, hidden)
        out = self.fc(out[:, -1, :])  # Get last time step's output
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)


# Setting HyperParameters

In [7]:
vocab_size=len(vocab)
embedding_dim=1024
hidden_dim=512
n_layer=2


# Initialize Model

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model=GRUPoemGenerator(vocab_size,embedding_dim,hidden_dim,n_layer).to(device)

cuda
GRU(1024, 512, num_layers=2, batch_first=True)


# Loss and Optimiser

In [9]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.005,weight_decay=1e-4)

# Training

In [10]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from tqdm import tqdm

def train_model(epochs):
    for epoch in range(epochs):
        total_loss = 0.0

        for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            batch_size = inputs.size(0)  # Get actual batch size
            
            hidden = model.init_hidden(batch_size)  # Initialize with correct batch size
            hidden = hidden.to(device)

            output, hidden = model(inputs, hidden)
            hidden = hidden.detach() 
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)  # Compute average loss
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss
            },
            "gru_model.pth"
        )


# Example usage (make sure to define these variables before calling the function)
# train_model(25, model, train_loader, optimizer, criterion, device)
train_model(20)

Epoch 1/20, Loss: 1.8837


Epoch 2/20, Loss: 1.6850


Epoch 3/20, Loss: 1.6204


Epoch 4/20, Loss: 1.5974


Epoch 5/20, Loss: 1.5833


...

Epoch 16/20, Loss: 1.5478


Epoch 17/20, Loss: 1.5474


Epoch 18/20, Loss: 1.5439


Epoch 19/20, Loss: 1.5482


Epoch 20/20, Loss: 1.5491


# Reload model and Train

In [28]:
import torch.nn.functional as F
def generate_poem(model, seed_text, length):
    model.eval()  # Set model to evaluation mode
    characters = list(seed_text)
    hidden = model.init_hidden(1).to(device)  # Ensure batch size = 1 for inference

    for _ in range(length):
        input_seq = torch.tensor([[vocab_to_int[character] for character in characters[-seq_length:]]], dtype=torch.long).to(device)
        
        # Ensure input_seq is 3D: (batch_size=1, seq_length, input_size)
        #input_seq = input_seq.unsqueeze(1)  # Add sequence dimension if missing
        
        output, hidden = model(input_seq, hidden)  # Pass input through GRU
        #output = output.squeeze(0)  # Remove batch dimension
        
        probabilities = F.softmax(output / 0.7, dim=-1)  # Apply softmax with temperature scaling
        next_char_idx = torch.multinomial(probabilities, 1).item()
        next_char = int_to_vocab[next_char_idx]
        
        characters.append(next_char)

    return ''.join(characters)




# Generate a poem
seed_text ="aañkh se duur na ho dil se utar jā.egā "
generated_poem = generate_poem(model, seed_text, length=1000)
print(generated_poem)

aañkh se duur na ho dil se utar jā.egā 
banā thā ye hair din kiye jhoñke rakh e tak saa.e 
kyā mere nākosh kā jo qosh balā nahīñ qayār 
soch jaa.e merī kamad-o-dast-e-mah-amb 
shikva ko dil ko vo ravād ke rakhte haiñ 
ki dil-e-dil-o-sharār legā kahāñ 
kohilb-e-raft kahī thī navāl murdan kahte hai 
musht ye kyā sitam kis dil-e-milan na karte haiñ 
vasl dard na sakte aise haiñ mahrūroñ se 
Dar ke kis se khā rahā hai jafā ne saath hai 
vo rah jaate haiñ vo lekh le sakte kare ko rah-e-tirīda-sar kaam 
maiñ ho gayā vo sar-e-nāz ho rakhte haiñ jo nahshubāñ kar 
bas-ki shab-e-nahīd se chal-e-naudsa-e-vosh nahīñ karte 
fikr-e-banā jaanā ki rasta tirī safar ko dhar aa.ī 
vo rahoñ ko vo vo se safar ke vo baḌas ho mire jalne se har dil ko rakh kar savā 
aa lagon kī rotī thoḌā rahe ho jaane 
dil ko so socho bohle ko koī na rahe hotā 
mile koī koshoke ho to kigle kahāñ o dost 
raho dard-e-vafā ho dar se to aaj ke sad-gavā 
kab lagtā hai mahd-e-dard se donoñ ko mashn ke safaj hotā 
mohabbat-e-dil ko

In [13]:
# import torch

In [14]:
# import torch


# checkpoint=torch.load('/kaggle/input/grumodel/gru_model.pth')
# model=GRUPoemGenerator(vocab_size,embedding_dim,hidden_dim,n_layer).to(device)
# optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch=checkpoint['epoch']
# loss=checkpoint['loss']
# train_model(25)